In [1]:
import numpy as np
import dgl
import scanpy as sc
import pandas as pd

import torch 
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


In [2]:
adata = sc.read('/home/ngr4/project/sccovid/data/processed/hbec_scv2.h5ad')

In [3]:
# pre-processing
adata_test = adata[adata.obs['split']=='test', :]
sc.tl.pca(adata_test, n_comps=50)
sc.external.pp.bbknn(adata_test, n_pcs=50, neighbors_within_batch=int(30/len(adata_test.obs['batch'].unique())))

/gpfs/ysm/project/dijk/ngr4/conda_envs/py38dev/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [4]:
g = dgl.from_scipy(adata_test.obsp['connectivities'])
g.ndata['h'] = torch.tensor(adata_test.obsm['X_pca']) # test with just PCA-coords as node features 

In [5]:
# add label
label_encoder = {'mock':0, 
                 'bystander_1dpi':1, 
                 'infected_1dpi':2, 
                 'bystander_2dpi':3, 
                 'infected_2dpi':4, 
                 'bystander_3dpi':5, 
                 'infected_3dpi':6}
g.ndata['label'] = torch.LongTensor(adata_test.obs['inf'].map(label_encoder).tolist())

In [6]:
import sys
sys.path.append('/home/ngr4/project/dev/graphtransformer/scripts/')
from layers import GraphTransformerLayer, GraphTransformerLayer_with_e

In [7]:
graphtransformer = GraphTransformerLayer(50, 50, 1)

In [8]:
graphtransformer(g, g.ndata['h'])

tensor([[-0.7363, -0.2228, -0.0875,  ..., -0.3864, -0.6127,  0.3716],
        [-0.3343, -1.9142, -0.0435,  ..., -1.1981,  0.1773, -2.5271],
        [-0.9305, -1.2824, -0.5805,  ...,  0.2150, -0.2188, -0.6366],
        ...,
        [ 0.8023, -1.0771, -1.1443,  ..., -0.8001, -0.0502,  0.3752],
        [ 1.3413,  0.0446,  0.0140,  ..., -0.2056, -0.0860, -0.1380],
        [ 0.6732,  0.3567,  0.2559,  ...,  0.7319,  0.5702,  0.1153]],
       grad_fn=<NativeBatchNormBackward>)

In [13]:
# try with random edge features?
g.edata['a'] = torch.rand((g.num_edges(), 50))
graphtransformer_with_e = GraphTransformerLayer_with_e(50, 50, 1)
graphtransformer_with_e(g, g.ndata['h'], g.edata['a']) # h_out, e_out

(tensor([[-0.7363, -0.1121, -0.0844,  ..., -1.7934, -1.1802,  0.4767],
         [-0.5928, -1.6743,  0.1341,  ..., -0.7764, -0.1976,  0.0663],
         [-1.0090, -1.1183, -0.5477,  ..., -0.3711,  0.6256, -0.5395],
         ...,
         [ 1.2062, -1.0328, -1.3876,  ...,  0.2576,  0.3574,  0.4905],
         [ 1.6420, -0.0639,  0.3728,  ...,  0.8759, -0.9098, -0.5620],
         [-0.0495,  0.2421,  0.5733,  ...,  0.1941,  0.5546, -0.0096]],
        grad_fn=<NativeBatchNormBackward>),
 tensor([[-0.1682,  0.5634,  0.2060,  ...,  0.0799, -0.1217, -0.2510],
         [ 0.2973, -0.1434,  0.1464,  ..., -0.1300, -0.5844,  0.1048],
         [-0.0295, -0.7795,  1.6222,  ..., -0.3397,  0.3698,  0.3646],
         ...,
         [ 1.5056,  2.0523,  2.2489,  ...,  1.8516, -0.7562,  1.0737],
         [ 0.9360, -0.6772,  2.9493,  ..., -0.6561,  0.4912,  1.0306],
         [ 0.4962,  0.7260, -0.6242,  ...,  0.0174, -0.0050,  0.4199]],
        grad_fn=<NativeBatchNormBackward>))

In [ ]:
# note positional encoding added after linear projection layer
## ideas: try UMAP/PHATE coords as alternatives to eigenvalues of laplacian
##          *note*: need PE = C^0 * embedding_coordinates + c^0 where C \in R^{dxk} and the projected node features have dim k 